In [1]:
#ppo algorithm single agent
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from sklearn.preprocessing import StandardScaler

# Function to load and normalize data
def load_and_normalize_data(train_file, test_file):
    df_train = pd.read_csv(train_file)
    df_test = pd.read_csv(test_file)

    df_train['timestamp'] = pd.to_datetime(df_train['timestamp'])
    df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])

    scaler = StandardScaler()
    columns_to_normalize = ['open', 'high', 'low', 'close', 'volume']

    df_train[columns_to_normalize] = scaler.fit_transform(df_train[columns_to_normalize])
    df_test[columns_to_normalize] = scaler.transform(df_test[columns_to_normalize])

    return df_train, df_test, scaler

# Single-Agent Trading Environment
class SingleAgentEnv(gym.Env):
    def __init__(self, data, window_size=10, initial_balance=10000, scaler=None):
        super(SingleAgentEnv, self).__init__()
        self.data = data
        self.window_size = window_size
        self.current_step = 0
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.position = 0  # 0 = neutral, 1 = long, -1 = short
        self.trades = []
        self.entry_price = 0
        self.log = []  # Log for detailed reporting
        self.scaler = scaler  # Store the scaler for inverse scaling

        # Action space: hold (0), buy (1), sell (2)
        self.action_space = spaces.Discrete(3)

        # Observation space: Stock prices (open, high, low, close, volume)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(window_size, 5), dtype=np.float32)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        self.position = 0
        self.balance = self.initial_balance
        self.trades = []
        self.entry_price = 0
        self.log = []  # Reset log

        # Log initial holdings
        self.log.append(f"Agent starts with 0 holdings (neutral position), Initial Balance: {self.balance}")
        return self._get_observation(), {}

    def _get_observation(self):
        obs = self.data.iloc[self.current_step:self.current_step + self.window_size][['open', 'high', 'low', 'close', 'volume']].values.astype(np.float32)
        return obs

    def inverse_scale_price(self, price):
        # Inverse transform the scaled price to get the original value (considering the 'close' column)
        inverse_scaled = self.scaler.inverse_transform([[0, 0, 0, price, 0]])[0][3]
        return inverse_scaled

    def step(self, action):
        reward = 0
        current_price = self.data.iloc[self.current_step]['close']
        original_price = self.inverse_scale_price(current_price)  # Get original (inverse-scaled) price

        # If agent buys
        if action == 1:
            if self.position == 0:  # Only buy if neutral
                self.position = 1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent buys at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Long")
            elif self.position == -1:  # Close short position
                reward = self.entry_price - current_price  # Scaled reward
                original_reward = self.inverse_scale_price(self.entry_price) - original_price
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes short at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        # If agent sells
        elif action == 2:
            if self.position == 0:  # Only sell if neutral
                self.position = -1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent sells (short) at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Short")
            elif self.position == 1:  # Close long position
                reward = current_price - self.entry_price  # Scaled reward
                original_reward = original_price - self.inverse_scale_price(self.entry_price)
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes long at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        self.current_step += 1
        terminated = self.current_step >= len(self.data) - self.window_size
        truncated = False
        return self._get_observation(), reward, terminated, truncated, {}

    def generate_report(self):
        print("\n--- Agent Report ---")
        for log in self.log:
            print(log)
        print(f"Final Balance: {self.balance}")
        print(f"Total Profit: {self.balance - self.initial_balance}")
        print(f"Number of Holdings (Long): {1 if self.position == 1 else 0}, Short: {1 if self.position == -1 else 0}")
        print("-" * 40)

# Function to calculate additional metrics
def calculate_metrics(trades, initial_balance, final_balance):
    # Total Profit
    total_profit = final_balance - initial_balance

    # Cumulative Return
    cumulative_return = (final_balance - initial_balance) / initial_balance

    # Win Rate
    positive_trades = [trade for trade in trades if trade > 0]
    win_rate = len(positive_trades) / len(trades) if trades else 0

    # Profit Factor
    gross_profit = sum(trade for trade in trades if trade > 0)
    gross_loss = -sum(trade for trade in trades if trade < 0)
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.inf

    # Sharpe Ratio
    returns = np.array(trades)
    mean_return = np.mean(returns)
    std_return = np.std(returns)
    sharpe_ratio = mean_return / std_return if std_return != 0 else 0

    # Sortino Ratio (using only downside standard deviation)
    downside_std = np.std([min(0, r) for r in returns])
    sortino_ratio = mean_return / downside_std if downside_std != 0 else 0

    # Maximum Drawdown
    balance_series = np.cumsum([initial_balance] + trades)  # Series of balance over time
    peak_balance = np.maximum.accumulate(balance_series)
    drawdowns = (peak_balance - balance_series) / peak_balance
    max_drawdown = np.max(drawdowns) if drawdowns.size > 0 else 0

    metrics = {
        "Total Profit": total_profit,
        "Cumulative Return": cumulative_return,
        "Win Rate": win_rate,
        "Profit Factor": profit_factor,
        "Sharpe Ratio": sharpe_ratio,
        "Sortino Ratio": sortino_ratio,
        "Maximum Drawdown": max_drawdown
    }
    return metrics

# Train and evaluate the model with all metrics
def train_and_evaluate():
    # Load and normalize the data
    train_file = 'NFLX_TRAINING.csv'
    test_file = 'NFLX_TESTING.csv'
    df_train_normalized, df_test_normalized, scaler = load_and_normalize_data(train_file, test_file)

    # Create the environment using the training data
    env_train = SingleAgentEnv(df_train_normalized, window_size=10, scaler=scaler)

    # Initialize the PPO model and train
    model = PPO("MlpPolicy", env_train, verbose=1)
    model.learn(total_timesteps=100000)

    # Test the model on the training data
    obs, _ = env_train.reset()
    done = False
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env_train.step(action)

    # Generate report for the training session
    env_train.generate_report()

    # Calculate and display metrics for the training period
    training_metrics = calculate_metrics(env_train.trades, env_train.initial_balance, env_train.balance)
    print("\n--- Training Metrics ---")
    for metric, value in training_metrics.items():
        print(f"{metric}: {value}")

    # Test the model on the testing data
    env_test = SingleAgentEnv(df_test_normalized, window_size=10, scaler=scaler)
    obs, _ = env_test.reset()
    done = False
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env_test.step(action)

    # Generate report for the testing session
    env_test.generate_report()

    # Calculate and display metrics for the testing period
    testing_metrics = calculate_metrics(env_test.trades, env_test.initial_balance, env_test.balance)
    print("\n--- Testing Metrics ---")
    for metric, value in testing_metrics.items():
        print(f"{metric}: {value}")

# Run the training and evaluation
train_and_evaluate()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 1896 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1499        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009333496 |
|    clip_fraction        | 0.0758      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -1.05       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0162     |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00813    |
|    value_loss         

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.14e+03    |
|    ep_rew_mean          | 14.9        |
| time/                   |             |
|    fps                  | 1336        |
|    iterations           | 12          |
|    time_elapsed         | 18          |
|    total_timesteps      | 24576       |
| train/                  |             |
|    approx_kl            | 0.010572189 |
|    clip_fraction        | 0.106       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.855      |
|    explained_variance   | 0.71        |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0368     |
|    n_updates            | 110         |
|    policy_gradient_loss | -0.00547    |
|    value_loss           | 0.00878     |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 7.14

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.14e+03    |
|    ep_rew_mean          | 34.6        |
| time/                   |             |
|    fps                  | 1321        |
|    iterations           | 22          |
|    time_elapsed         | 34          |
|    total_timesteps      | 45056       |
| train/                  |             |
|    approx_kl            | 0.009380452 |
|    clip_fraction        | 0.0942      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.797      |
|    explained_variance   | 0.631       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.00885    |
|    n_updates            | 210         |
|    policy_gradient_loss | -0.00436    |
|    value_loss           | 0.00825     |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 7.14

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.14e+03    |
|    ep_rew_mean          | 45.2        |
| time/                   |             |
|    fps                  | 1322        |
|    iterations           | 32          |
|    time_elapsed         | 49          |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.014186889 |
|    clip_fraction        | 0.11        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.738      |
|    explained_variance   | 0.522       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.00565    |
|    n_updates            | 310         |
|    policy_gradient_loss | -0.00558    |
|    value_loss           | 0.00781     |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 7.14

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.14e+03    |
|    ep_rew_mean          | 52.6        |
| time/                   |             |
|    fps                  | 1306        |
|    iterations           | 42          |
|    time_elapsed         | 65          |
|    total_timesteps      | 86016       |
| train/                  |             |
|    approx_kl            | 0.007000295 |
|    clip_fraction        | 0.0833      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.58       |
|    explained_variance   | 0.667       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.00592    |
|    n_updates            | 410         |
|    policy_gradient_loss | -0.00577    |
|    value_loss           | 0.00952     |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.14e+


--- Agent Report ---
Agent starts with 0 holdings (neutral position), Initial Balance: 10000
Agent sells (short) at 604.88, Current Balance: 10000, Holdings: 1 Short
Agent closes short at 550.3, profit: 54.58000000000004, Current Balance: 10054.58, Holdings: 0
Agent buys at 555.425, Current Balance: 10054.58, Holdings: 1 Long
Agent closes long at 557.99, profit: 2.5650000000000546, Current Balance: 10057.145, Holdings: 0
Agent sells (short) at 557.2253, Current Balance: 10057.145, Holdings: 1 Short
Agent closes short at 553.29, profit: 3.935299999999984, Current Balance: 10061.0803, Holdings: 0
Agent sells (short) at 554.7, Current Balance: 10061.0803, Holdings: 1 Short
Agent closes short at 539.665, profit: 15.035000000000082, Current Balance: 10076.1153, Holdings: 0
Agent buys at 541.06, Current Balance: 10076.1153, Holdings: 1 Long
Agent closes long at 540.88, profit: -0.17999999999994998, Current Balance: 10075.9353, Holdings: 0
Agent sells (short) at 540.85, Current Balance: 1007

In [2]:
#dqn algorithm single agent
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import DQN
from sklearn.preprocessing import StandardScaler

# Function to load and normalize data
def load_and_normalize_data(train_file, test_file):
    df_train = pd.read_csv(train_file)
    df_test = pd.read_csv(test_file)

    df_train['timestamp'] = pd.to_datetime(df_train['timestamp'])
    df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])

    scaler = StandardScaler()
    columns_to_normalize = ['open', 'high', 'low', 'close', 'volume']

    df_train[columns_to_normalize] = scaler.fit_transform(df_train[columns_to_normalize])
    df_test[columns_to_normalize] = scaler.transform(df_test[columns_to_normalize])

    return df_train, df_test, scaler

# Single-Agent Trading Environment
class SingleAgentEnv(gym.Env):
    def __init__(self, data, window_size=10, initial_balance=10000, scaler=None):
        super(SingleAgentEnv, self).__init__()
        self.data = data
        self.window_size = window_size
        self.current_step = 0
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.position = 0  # 0 = neutral, 1 = long, -1 = short
        self.trades = []
        self.entry_price = 0
        self.log = []  # Log for detailed reporting
        self.scaler = scaler  # Store the scaler for inverse scaling

        # Action space: hold (0), buy (1), sell (2)
        self.action_space = spaces.Discrete(3)

        # Observation space: Stock prices (open, high, low, close, volume)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(window_size, 5), dtype=np.float32)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        self.position = 0
        self.balance = self.initial_balance
        self.trades = []
        self.entry_price = 0
        self.log = []  # Reset log

        # Log initial holdings
        self.log.append(f"Agent starts with 0 holdings (neutral position), Initial Balance: {self.balance}")
        return self._get_observation(), {}

    def _get_observation(self):
        obs = self.data.iloc[self.current_step:self.current_step + self.window_size][['open', 'high', 'low', 'close', 'volume']].values.astype(np.float32)
        return obs

    def inverse_scale_price(self, price):
        # Inverse transform the scaled price to get the original value (considering the 'close' column)
        inverse_scaled = self.scaler.inverse_transform([[0, 0, 0, price, 0]])[0][3]
        return inverse_scaled

    def step(self, action):
        reward = 0
        current_price = self.data.iloc[self.current_step]['close']
        original_price = self.inverse_scale_price(current_price)  # Get original (inverse-scaled) price

        # If agent buys
        if action == 1:
            if self.position == 0:  # Only buy if neutral
                self.position = 1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent buys at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Long")
            elif self.position == -1:  # Close short position
                reward = self.entry_price - current_price  # Scaled reward
                original_reward = self.inverse_scale_price(self.entry_price) - original_price
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes short at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        # If agent sells
        elif action == 2:
            if self.position == 0:  # Only sell if neutral
                self.position = -1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent sells (short) at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Short")
            elif self.position == 1:  # Close long position
                reward = current_price - self.entry_price  # Scaled reward
                original_reward = original_price - self.inverse_scale_price(self.entry_price)
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes long at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        self.current_step += 1
        terminated = self.current_step >= len(self.data) - self.window_size
        truncated = False
        return self._get_observation(), reward, terminated, truncated, {}

    def generate_report(self):
        print("\n--- Agent Report ---")
        for log in self.log:
            print(log)
        print(f"Final Balance: {self.balance}")
        print(f"Total Profit: {self.balance - self.initial_balance}")
        print(f"Number of Holdings (Long): {1 if self.position == 1 else 0}, Short: {1 if self.position == -1 else 0}")
        print("-" * 40)

# Function to calculate additional metrics
def calculate_metrics(trades, initial_balance, final_balance):
    # Total Profit
    total_profit = final_balance - initial_balance

    # Cumulative Return
    cumulative_return = (final_balance - initial_balance) / initial_balance

    # Win Rate
    positive_trades = [trade for trade in trades if trade > 0]
    win_rate = len(positive_trades) / len(trades) if trades else 0

    # Profit Factor
    gross_profit = sum(trade for trade in trades if trade > 0)
    gross_loss = -sum(trade for trade in trades if trade < 0)
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.inf

    # Sharpe Ratio
    returns = np.array(trades)
    mean_return = np.mean(returns)
    std_return = np.std(returns)
    sharpe_ratio = mean_return / std_return if std_return != 0 else 0

    # Sortino Ratio (using only downside standard deviation)
    downside_std = np.std([min(0, r) for r in returns])
    sortino_ratio = mean_return / downside_std if downside_std != 0 else 0

    # Maximum Drawdown
    balance_series = np.cumsum([initial_balance] + trades)  # Series of balance over time
    peak_balance = np.maximum.accumulate(balance_series)
    drawdowns = (peak_balance - balance_series) / peak_balance
    max_drawdown = np.max(drawdowns) if drawdowns.size > 0 else 0

    metrics = {
        "Total Profit": total_profit,
        "Cumulative Return": cumulative_return,
        "Win Rate": win_rate,
        "Profit Factor": profit_factor,
        "Sharpe Ratio": sharpe_ratio,
        "Sortino Ratio": sortino_ratio,
        "Maximum Drawdown": max_drawdown
    }
    return metrics

# Train and evaluate the model with all metrics
def train_and_evaluate():
    # Load and normalize the data
    train_file = 'NFLX_TRAINING.csv'
    test_file = 'NFLX_TESTING.csv'
    df_train_normalized, df_test_normalized, scaler = load_and_normalize_data(train_file, test_file)

    # Create the environment using the training data
    env_train = SingleAgentEnv(df_train_normalized, window_size=10, scaler=scaler)

    # Initialize the DQN model and train
    model = DQN("MlpPolicy", env_train, verbose=1)
    model.learn(total_timesteps=100000)

    # Test the model on the training data
    obs, _ = env_train.reset()
    done = False
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env_train.step(action)

    # Generate report for the training session
    env_train.generate_report()

    # Calculate and display metrics for the training period
    training_metrics = calculate_metrics(env_train.trades, env_train.initial_balance, env_train.balance)
    print("\n--- Training Metrics ---")
    for metric, value in training_metrics.items():
        print(f"{metric}: {value}")

    # Test the model on the testing data
    env_test = SingleAgentEnv(df_test_normalized, window_size=10, scaler=scaler)
    obs, _ = env_test.reset()
    done = False
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env_test.step(action)

    # Generate report for the testing session
    env_test.generate_report()

    # Calculate and display metrics for the testing period
    testing_metrics = calculate_metrics(env_test.trades, env_test.initial_balance, env_test.balance)
    print("\n--- Testing Metrics ---")
    for metric, value in testing_metrics.items():
        print(f"{metric}: {value}")

# Run the training and evaluation
train_and_evaluate()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 7.14e+03 |
|    ep_rew_mean      | 3.23     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1100     |
|    time_elapsed     | 25       |
|    total_timesteps  | 28576    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000634 |
|    n_updates        | 7118     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 7.14e+03 |
|    ep_rew_mean      | 2.31     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1124     |
|    time_elapsed     | 50       |
|    total_timesteps  | 57152    |
| train/              |        


--- Agent Report ---
Agent starts with 0 holdings (neutral position), Initial Balance: 10000
Agent buys at 604.88, Current Balance: 10000, Holdings: 1 Long
Agent closes long at 592.38, profit: -12.5, Current Balance: 9987.5, Holdings: 0
Agent buys at 594.88, Current Balance: 9987.5, Holdings: 1 Long
Agent closes long at 596.38, profit: 1.5, Current Balance: 9989.0, Holdings: 0
Agent buys at 596.4, Current Balance: 9989.0, Holdings: 1 Long
Agent closes long at 592.6, profit: -3.7999999999999545, Current Balance: 9985.2, Holdings: 0
Agent sells (short) at 585.32, Current Balance: 9985.2, Holdings: 1 Short
Agent closes short at 582.02, profit: 3.300000000000068, Current Balance: 9988.5, Holdings: 0
Agent sells (short) at 587.415, Current Balance: 9988.5, Holdings: 1 Short
Agent closes short at 591.2101, profit: -3.7951000000000477, Current Balance: 9984.7049, Holdings: 0
Agent buys at 592.0, Current Balance: 9984.7049, Holdings: 1 Long
Agent closes long at 590.66, profit: -1.340000000000

In [3]:
#a2c algorithm single agent
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import A2C
from sklearn.preprocessing import StandardScaler

# Function to load and normalize data
def load_and_normalize_data(train_file, test_file):
    df_train = pd.read_csv(train_file)
    df_test = pd.read_csv(test_file)

    df_train['timestamp'] = pd.to_datetime(df_train['timestamp'])
    df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])

    scaler = StandardScaler()
    columns_to_normalize = ['open', 'high', 'low', 'close', 'volume']

    df_train[columns_to_normalize] = scaler.fit_transform(df_train[columns_to_normalize])
    df_test[columns_to_normalize] = scaler.transform(df_test[columns_to_normalize])

    return df_train, df_test, scaler

# Single-Agent Trading Environment
class SingleAgentEnv(gym.Env):
    def __init__(self, data, window_size=10, initial_balance=10000, scaler=None):
        super(SingleAgentEnv, self).__init__()
        self.data = data
        self.window_size = window_size
        self.current_step = 0
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.position = 0  # 0 = neutral, 1 = long, -1 = short
        self.trades = []
        self.entry_price = 0
        self.log = []  # Log for detailed reporting
        self.scaler = scaler  # Store the scaler for inverse scaling

        # Action space: hold (0), buy (1), sell (2)
        self.action_space = spaces.Discrete(3)

        # Observation space: Stock prices (open, high, low, close, volume)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(window_size, 5), dtype=np.float32)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        self.position = 0
        self.balance = self.initial_balance
        self.trades = []
        self.entry_price = 0
        self.log = []  # Reset log

        # Log initial holdings
        self.log.append(f"Agent starts with 0 holdings (neutral position), Initial Balance: {self.balance}")
        return self._get_observation(), {}

    def _get_observation(self):
        obs = self.data.iloc[self.current_step:self.current_step + self.window_size][['open', 'high', 'low', 'close', 'volume']].values.astype(np.float32)
        return obs

    def inverse_scale_price(self, price):
        # Inverse transform the scaled price to get the original value (considering the 'close' column)
        inverse_scaled = self.scaler.inverse_transform([[0, 0, 0, price, 0]])[0][3]
        return inverse_scaled

    def step(self, action):
        reward = 0
        current_price = self.data.iloc[self.current_step]['close']
        original_price = self.inverse_scale_price(current_price)  # Get original (inverse-scaled) price

        # If agent buys
        if action == 1:
            if self.position == 0:  # Only buy if neutral
                self.position = 1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent buys at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Long")
            elif self.position == -1:  # Close short position
                reward = self.entry_price - current_price  # Scaled reward
                original_reward = self.inverse_scale_price(self.entry_price) - original_price
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes short at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        # If agent sells
        elif action == 2:
            if self.position == 0:  # Only sell if neutral
                self.position = -1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent sells (short) at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Short")
            elif self.position == 1:  # Close long position
                reward = current_price - self.entry_price  # Scaled reward
                original_reward = original_price - self.inverse_scale_price(self.entry_price)
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes long at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        self.current_step += 1
        terminated = self.current_step >= len(self.data) - self.window_size
        truncated = False
        return self._get_observation(), reward, terminated, truncated, {}

    def generate_report(self):
        print("\n--- Agent Report ---")
        for log in self.log:
            print(log)
        print(f"Final Balance: {self.balance}")
        print(f"Total Profit: {self.balance - self.initial_balance}")
        print(f"Number of Holdings (Long): {1 if self.position == 1 else 0}, Short: {1 if self.position == -1 else 0}")
        print("-" * 40)

# Function to calculate additional metrics
def calculate_metrics(trades, initial_balance, final_balance):
    # Total Profit
    total_profit = final_balance - initial_balance

    # Cumulative Return
    cumulative_return = (final_balance - initial_balance) / initial_balance

    # Win Rate
    positive_trades = [trade for trade in trades if trade > 0]
    win_rate = len(positive_trades) / len(trades) if trades else 0

    # Profit Factor
    gross_profit = sum(trade for trade in trades if trade > 0)
    gross_loss = -sum(trade for trade in trades if trade < 0)
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.inf

    # Sharpe Ratio
    returns = np.array(trades)
    mean_return = np.mean(returns)
    std_return = np.std(returns)
    sharpe_ratio = mean_return / std_return if std_return != 0 else 0

    # Sortino Ratio (using only downside standard deviation)
    downside_std = np.std([min(0, r) for r in returns])
    sortino_ratio = mean_return / downside_std if downside_std != 0 else 0

    # Maximum Drawdown
    balance_series = np.cumsum([initial_balance] + trades)  # Series of balance over time
    peak_balance = np.maximum.accumulate(balance_series)
    drawdowns = (peak_balance - balance_series) / peak_balance
    max_drawdown = np.max(drawdowns) if drawdowns.size > 0 else 0

    metrics = {
        "Total Profit": total_profit,
        "Cumulative Return": cumulative_return,
        "Win Rate": win_rate,
        "Profit Factor": profit_factor,
        "Sharpe Ratio": sharpe_ratio,
        "Sortino Ratio": sortino_ratio,
        "Maximum Drawdown": max_drawdown
    }
    return metrics

# Train and evaluate the model with action logging for debugging
def train_and_evaluate():
    # Load and normalize the data
    train_file = 'NFLX_TRAINING.csv'
    test_file = 'NFLX_TESTING.csv'
    df_train_normalized, df_test_normalized, scaler = load_and_normalize_data(train_file, test_file)

    # Create the environment using the training data
    env_train = SingleAgentEnv(df_train_normalized, window_size=10, scaler=scaler)

    # Initialize the A2C model and train with more timesteps
    model = A2C("MlpPolicy", env_train, verbose=1)
    model.learn(total_timesteps=500000)  # Increased timesteps

    # Testing on the training data with action logging
    obs, _ = env_train.reset()
    done = False
    print("\n--- Training Session ---")
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env_train.step(action)
        # Log each action and reward for debugging
        print(f"Step: {env_train.current_step}, Action: {action}, Reward: {reward}")

    # Generate report and metrics for the training session
    env_train.generate_report()
    training_metrics = calculate_metrics(env_train.trades, env_train.initial_balance, env_train.balance)
    print("\n--- Training Metrics ---")
    for metric, value in training_metrics.items():
        print(f"{metric}: {value}")

    # Testing on the testing data with action logging
    env_test = SingleAgentEnv(df_test_normalized, window_size=10, scaler=scaler)
    obs, _ = env_test.reset()
    done = False
    print("\n--- Testing Session ---")
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env_test.step(action)
        # Log each action and reward for debugging
        print(f"Step: {env_test.current_step}, Action: {action}, Reward: {reward}")

    # Generate report and metrics for the testing session
    env_test.generate_report()
    testing_metrics = calculate_metrics(env_test.trades, env_test.initial_balance, env_test.balance)
    print("\n--- Testing Metrics ---")
    for metric, value in testing_metrics.items():
        print(f"{metric}: {value}")



# Run the training and evaluation
train_and_evaluate()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| time/                 |          |
|    fps                | 1166     |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -0.748   |
|    explained_variance | -32.2    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 0.0214   |
|    value_loss         | 0.000801 |
------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 1201      |
|    iterations         | 200       |
|    time_elapsed       | 0         |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -1.01     |
|    explained_variance | -2.27e+03 |
|    learning_rate      | 0.0007    |
|    n_u

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.14e+03 |
|    ep_rew_mean        | -1.38    |
| time/                 |          |
|    fps                | 1016     |
|    iterations         | 1700     |
|    time_elapsed       | 8        |
|    total_timesteps    | 8500     |
| train/                |          |
|    entropy_loss       | -0.531   |
|    explained_variance | -32.5    |
|    learning_rate      | 0.0007   |
|    n_updates          | 1699     |
|    policy_loss        | -0.0183  |
|    value_loss         | 0.00177  |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.14e+03 |
|    ep_rew_mean        | -1.38    |
| time/                 |          |
|    fps                | 1012     |
|    iterations         | 1800     |
|    time_elapsed       | 8        |
|    total_timesteps    | 9000     |
| train/                |          |
|

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 7.14e+03  |
|    ep_rew_mean        | 0.411     |
| time/                 |           |
|    fps                | 1000      |
|    iterations         | 3000      |
|    time_elapsed       | 14        |
|    total_timesteps    | 15000     |
| train/                |           |
|    entropy_loss       | -0.151    |
|    explained_variance | -10.4     |
|    learning_rate      | 0.0007    |
|    n_updates          | 2999      |
|    policy_loss        | -6.29e-05 |
|    value_loss         | 1.28e-05  |
-------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.14e+03 |
|    ep_rew_mean        | 0.411    |
| time/                 |          |
|    fps                | 999      |
|    iterations         | 3100     |
|    time_elapsed       | 15       |
|    total_timesteps    | 15500    |
| train/             

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.14e+03 |
|    ep_rew_mean        | -0.74    |
| time/                 |          |
|    fps                | 988      |
|    iterations         | 4300     |
|    time_elapsed       | 21       |
|    total_timesteps    | 21500    |
| train/                |          |
|    entropy_loss       | -0.538   |
|    explained_variance | 0.442    |
|    learning_rate      | 0.0007   |
|    n_updates          | 4299     |
|    policy_loss        | -0.00428 |
|    value_loss         | 6.42e-05 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.14e+03 |
|    ep_rew_mean        | -0.74    |
| time/                 |          |
|    fps                | 987      |
|    iterations         | 4400     |
|    time_elapsed       | 22       |
|    total_timesteps    | 22000    |
| train/                |          |
|

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 7.14e+03  |
|    ep_rew_mean        | -0.74     |
| time/                 |           |
|    fps                | 977       |
|    iterations         | 5600      |
|    time_elapsed       | 28        |
|    total_timesteps    | 28000     |
| train/                |           |
|    entropy_loss       | -0.0356   |
|    explained_variance | -29.1     |
|    learning_rate      | 0.0007    |
|    n_updates          | 5599      |
|    policy_loss        | -2.12e-06 |
|    value_loss         | 4.78e-07  |
-------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.14e+03 |
|    ep_rew_mean        | -0.74    |
| time/                 |          |
|    fps                | 977      |
|    iterations         | 5700     |
|    time_elapsed       | 29       |
|    total_timesteps    | 28500    |
| train/             

...

Step: 1942, Action: 2, Reward: 0
Step: 1943, Action: 2, Reward: 0
Step: 1944, Action: 2, Reward: 0
Step: 1945, Action: 2, Reward: 0
Step: 1946, Action: 2, Reward: 0
Step: 1947, Action: 2, Reward: 0
Step: 1948, Action: 2, Reward: 0
Step: 1949, Action: 2, Reward: 0
Step: 1950, Action: 2, Reward: 0
Step: 1951, Action: 2, Reward: 0
Step: 1952, Action: 2, Reward: 0
Step: 1953, Action: 2, Reward: 0
Step: 1954, Action: 2, Reward: 0
Step: 1955, Action: 2, Reward: 0
Step: 1956, Action: 2, Reward: 0
Step: 1957, Action: 2, Reward: 0
Step: 1958, Action: 2, Reward: 0
Step: 1959, Action: 2, Reward: 0
Step: 1960, Action: 2, Reward: 0
Step: 1961, Action: 2, Reward: 0
Step: 1962, Action: 2, Reward: 0
Step: 1963, Action: 2, Reward: 0
Step: 1964, Action: 2, Reward: 0
Step: 1965, Action: 2, Reward: 0
Step: 1966, Action: 2, Reward: 0
Step: 1967, Action: 2, Reward: 0
Step: 1968, Action: 2, Reward: 0
Step: 1969, Action: 2, Reward: 0
Step: 1970, Action: 2, Reward: 0
Step: 1971, Action: 2, Reward: 0
Step: 1972

Step: 2378, Action: 2, Reward: 0
Step: 2379, Action: 2, Reward: 0
Step: 2380, Action: 2, Reward: 0
Step: 2381, Action: 2, Reward: 0
Step: 2382, Action: 2, Reward: 0
Step: 2383, Action: 2, Reward: 0
Step: 2384, Action: 2, Reward: 0
Step: 2385, Action: 2, Reward: 0
Step: 2386, Action: 2, Reward: 0
Step: 2387, Action: 2, Reward: 0
Step: 2388, Action: 2, Reward: 0
Step: 2389, Action: 2, Reward: 0
Step: 2390, Action: 2, Reward: 0
Step: 2391, Action: 2, Reward: 0
Step: 2392, Action: 2, Reward: 0
Step: 2393, Action: 2, Reward: 0
Step: 2394, Action: 2, Reward: 0
Step: 2395, Action: 2, Reward: 0
Step: 2396, Action: 2, Reward: 0
Step: 2397, Action: 2, Reward: 0
Step: 2398, Action: 2, Reward: 0
Step: 2399, Action: 2, Reward: 0
Step: 2400, Action: 2, Reward: 0
Step: 2401, Action: 2, Reward: 0
Step: 2402, Action: 2, Reward: 0
Step: 2403, Action: 2, Reward: 0
Step: 2404, Action: 2, Reward: 0
Step: 2405, Action: 2, Reward: 0
Step: 2406, Action: 2, Reward: 0
Step: 2407, Action: 2, Reward: 0
Step: 2408

Step: 2819, Action: 2, Reward: 0
Step: 2820, Action: 2, Reward: 0
Step: 2821, Action: 2, Reward: 0
Step: 2822, Action: 2, Reward: 0
Step: 2823, Action: 2, Reward: 0
Step: 2824, Action: 2, Reward: 0
Step: 2825, Action: 2, Reward: 0
Step: 2826, Action: 2, Reward: 0
Step: 2827, Action: 2, Reward: 0
Step: 2828, Action: 2, Reward: 0
Step: 2829, Action: 2, Reward: 0
Step: 2830, Action: 2, Reward: 0
Step: 2831, Action: 2, Reward: 0
Step: 2832, Action: 2, Reward: 0
Step: 2833, Action: 2, Reward: 0
Step: 2834, Action: 2, Reward: 0
Step: 2835, Action: 2, Reward: 0
Step: 2836, Action: 2, Reward: 0
Step: 2837, Action: 2, Reward: 0
Step: 2838, Action: 2, Reward: 0
Step: 2839, Action: 2, Reward: 0
Step: 2840, Action: 2, Reward: 0
Step: 2841, Action: 2, Reward: 0
Step: 2842, Action: 2, Reward: 0
Step: 2843, Action: 2, Reward: 0
Step: 2844, Action: 2, Reward: 0
Step: 2845, Action: 2, Reward: 0
Step: 2846, Action: 2, Reward: 0
Step: 2847, Action: 2, Reward: 0
Step: 2848, Action: 2, Reward: 0
Step: 2849

Step: 3255, Action: 2, Reward: 0
Step: 3256, Action: 2, Reward: 0
Step: 3257, Action: 2, Reward: 0
Step: 3258, Action: 2, Reward: 0
Step: 3259, Action: 2, Reward: 0
Step: 3260, Action: 2, Reward: 0
Step: 3261, Action: 2, Reward: 0
Step: 3262, Action: 2, Reward: 0
Step: 3263, Action: 2, Reward: 0
Step: 3264, Action: 2, Reward: 0
Step: 3265, Action: 2, Reward: 0
Step: 3266, Action: 2, Reward: 0
Step: 3267, Action: 2, Reward: 0
Step: 3268, Action: 2, Reward: 0
Step: 3269, Action: 2, Reward: 0
Step: 3270, Action: 2, Reward: 0
Step: 3271, Action: 2, Reward: 0
Step: 3272, Action: 2, Reward: 0
Step: 3273, Action: 2, Reward: 0
Step: 3274, Action: 2, Reward: 0
Step: 3275, Action: 2, Reward: 0
Step: 3276, Action: 2, Reward: 0
Step: 3277, Action: 2, Reward: 0
Step: 3278, Action: 2, Reward: 0
Step: 3279, Action: 2, Reward: 0
Step: 3280, Action: 2, Reward: 0
Step: 3281, Action: 2, Reward: 0
Step: 3282, Action: 2, Reward: 0
Step: 3283, Action: 2, Reward: 0
Step: 3284, Action: 2, Reward: 0
Step: 3285

Step: 3693, Action: 2, Reward: 0
Step: 3694, Action: 2, Reward: 0
Step: 3695, Action: 2, Reward: 0
Step: 3696, Action: 2, Reward: 0
Step: 3697, Action: 2, Reward: 0
Step: 3698, Action: 2, Reward: 0
Step: 3699, Action: 2, Reward: 0
Step: 3700, Action: 2, Reward: 0
Step: 3701, Action: 2, Reward: 0
Step: 3702, Action: 2, Reward: 0
Step: 3703, Action: 2, Reward: 0
Step: 3704, Action: 2, Reward: 0
Step: 3705, Action: 2, Reward: 0
Step: 3706, Action: 2, Reward: 0
Step: 3707, Action: 2, Reward: 0
Step: 3708, Action: 2, Reward: 0
Step: 3709, Action: 2, Reward: 0
Step: 3710, Action: 2, Reward: 0
Step: 3711, Action: 2, Reward: 0
Step: 3712, Action: 2, Reward: 0
Step: 3713, Action: 2, Reward: 0
Step: 3714, Action: 2, Reward: 0
Step: 3715, Action: 2, Reward: 0
Step: 3716, Action: 2, Reward: 0
Step: 3717, Action: 2, Reward: 0
Step: 3718, Action: 2, Reward: 0
Step: 3719, Action: 2, Reward: 0
Step: 3720, Action: 2, Reward: 0
Step: 3721, Action: 2, Reward: 0
Step: 3722, Action: 2, Reward: 0
Step: 3723

In [4]:
#ensemble learning single agent
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO, DQN, A2C
from sklearn.preprocessing import StandardScaler
from collections import Counter

# Function to load and normalize data
def load_and_normalize_data(train_file, test_file):
    df_train = pd.read_csv(train_file)
    df_test = pd.read_csv(test_file)

    df_train['timestamp'] = pd.to_datetime(df_train['timestamp'])
    df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])

    scaler = StandardScaler()
    columns_to_normalize = ['open', 'high', 'low', 'close', 'volume']

    df_train[columns_to_normalize] = scaler.fit_transform(df_train[columns_to_normalize])
    df_test[columns_to_normalize] = scaler.transform(df_test[columns_to_normalize])

    return df_train, df_test, scaler

# Single-Agent Trading Environment
class SingleAgentEnv(gym.Env):
    def __init__(self, data, window_size=10, initial_balance=10000, scaler=None):
        super(SingleAgentEnv, self).__init__()
        self.data = data
        self.window_size = window_size
        self.current_step = 0
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.position = 0  # 0 = neutral, 1 = long, -1 = short
        self.trades = []
        self.entry_price = 0
        self.log = []  # Log for detailed reporting
        self.scaler = scaler  # Store the scaler for inverse scaling

        # Action space: hold (0), buy (1), sell (2)
        self.action_space = spaces.Discrete(3)

        # Observation space: Stock prices (open, high, low, close, volume)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(window_size, 5), dtype=np.float32)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        self.position = 0
        self.balance = self.initial_balance
        self.trades = []
        self.entry_price = 0
        self.log = []  # Reset log

        # Log initial holdings
        self.log.append(f"Agent starts with 0 holdings (neutral position), Initial Balance: {self.balance}")
        return self._get_observation(), {}

    def _get_observation(self):
        obs = self.data.iloc[self.current_step:self.current_step + self.window_size][['open', 'high', 'low', 'close', 'volume']].values.astype(np.float32)
        return obs

    def inverse_scale_price(self, price):
        # Inverse transform the scaled price to get the original value (considering the 'close' column)
        inverse_scaled = self.scaler.inverse_transform([[0, 0, 0, price, 0]])[0][3]
        return inverse_scaled

    def step(self, action):
        reward = 0
        current_price = self.data.iloc[self.current_step]['close']
        original_price = self.inverse_scale_price(current_price)  # Get original (inverse-scaled) price

        # If agent buys
        if action == 1:
            if self.position == 0:  # Only buy if neutral
                self.position = 1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent buys at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Long")
            elif self.position == -1:  # Close short position
                reward = self.entry_price - current_price  # Scaled reward
                original_reward = self.inverse_scale_price(self.entry_price) - original_price
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes short at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        # If agent sells
        elif action == 2:
            if self.position == 0:  # Only sell if neutral
                self.position = -1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent sells (short) at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Short")
            elif self.position == 1:  # Close long position
                reward = current_price - self.entry_price  # Scaled reward
                original_reward = original_price - self.inverse_scale_price(self.entry_price)
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes long at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        self.current_step += 1
        terminated = self.current_step >= len(self.data) - self.window_size
        truncated = False
        return self._get_observation(), reward, terminated, truncated, {}

    def generate_report(self):
        print("\n--- Agent Report ---")
        for log in self.log:
            print(log)
        print(f"Final Balance: {self.balance}")
        print(f"Total Profit: {self.balance - self.initial_balance}")
        print(f"Number of Holdings (Long): {1 if self.position == 1 else 0}, Short: {1 if self.position == -1 else 0}")
        print("-" * 40)

# Function to calculate metrics
def calculate_metrics(trades, initial_balance, final_balance):
    # Total Profit
    total_profit = final_balance - initial_balance

    # Cumulative Return
    cumulative_return = (final_balance - initial_balance) / initial_balance

    # Win Rate
    positive_trades = [trade for trade in trades if trade > 0]
    win_rate = len(positive_trades) / len(trades) if trades else 0

    # Profit Factor
    gross_profit = sum(trade for trade in trades if trade > 0)
    gross_loss = -sum(trade for trade in trades if trade < 0)
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.inf

    # Sharpe Ratio
    returns = np.array(trades)
    mean_return = np.mean(returns)
    std_return = np.std(returns)
    sharpe_ratio = mean_return / std_return if std_return != 0 else 0

    # Sortino Ratio (using only downside standard deviation)
    downside_std = np.std([min(0, r) for r in returns])
    sortino_ratio = mean_return / downside_std if downside_std != 0 else 0

    # Maximum Drawdown
    balance_series = np.cumsum([initial_balance] + trades)  # Series of balance over time
    peak_balance = np.maximum.accumulate(balance_series)
    drawdowns = (peak_balance - balance_series) / peak_balance
    max_drawdown = np.max(drawdowns) if drawdowns.size > 0 else 0

    metrics = {
        "Total Profit": total_profit,
        "Cumulative Return": cumulative_return,
        "Win Rate": win_rate,
        "Profit Factor": profit_factor,
        "Sharpe Ratio": sharpe_ratio,
        "Sortino Ratio": sortino_ratio,
        "Maximum Drawdown": max_drawdown
    }
    return metrics

# Ensemble model function
def ensemble_predict(actions):
    # Convert numpy arrays to integers for each action
    actions = [int(action) for action in actions]
    # Perform a majority vote among the actions (hold, buy, sell)
    action_counts = Counter(actions)
    return action_counts.most_common(1)[0][0]

# Train and evaluate the ensemble model
def train_and_evaluate():
    # Load and normalize the data
    train_file = 'NFLX_TRAINING.csv'
    test_file = 'NFLX_TESTING.csv'
    df_train_normalized, df_test_normalized, scaler = load_and_normalize_data(train_file, test_file)

    # Create the environment using the training data
    env_train = SingleAgentEnv(df_train_normalized, window_size=10, scaler=scaler)

    # Initialize each model and train them separately
    ppo_model = PPO("MlpPolicy", env_train, verbose=1)
    dqn_model = DQN("MlpPolicy", env_train, verbose=1)
    a2c_model = A2C("MlpPolicy", env_train, verbose=1)

    # Train each model
    ppo_model.learn(total_timesteps=50000)
    dqn_model.learn(total_timesteps=50000)
    a2c_model.learn(total_timesteps=50000)

    # Test the ensemble model on the training data
    obs, _ = env_train.reset()
    done = False
    while not done:
        # Get predictions from each model
        ppo_action, _ = ppo_model.predict(obs)
        dqn_action, _ = dqn_model.predict(obs)
        a2c_action, _ = a2c_model.predict(obs)

        # Aggregate the actions through majority voting
        final_action = ensemble_predict([ppo_action, dqn_action, a2c_action])

        # Step the environment with the final action
        obs, reward, done, truncated, info = env_train.step(final_action)

    # Calculate and display training metrics
    training_metrics = calculate_metrics(env_train.trades, env_train.initial_balance, env_train.balance)
    print("\n--- Training Metrics ---")
    for metric, value in training_metrics.items():
        print(f"{metric}: {value}")

    # Test the ensemble model on the testing data
    env_test = SingleAgentEnv(df_test_normalized, window_size=10, scaler=scaler)
    obs, _ = env_test.reset()
    done = False
    while not done:
        # Get predictions from each model
        ppo_action, _ = ppo_model.predict(obs)
        dqn_action, _ = dqn_model.predict(obs)
        a2c_action, _ = a2c_model.predict(obs)

        # Aggregate the actions through majority voting
        final_action = ensemble_predict([ppo_action, dqn_action, a2c_action])

        # Step the environment with the final action
        obs, reward, done, truncated, info = env_test.step(final_action)

    # Generate report for the testing session
    env_test.generate_report()

    # Calculate and display testing metrics
    testing_metrics = calculate_metrics(env_test.trades, env_test.initial_balance, env_test.balance)
    print("\n--- Testing Metrics ---")
    for metric, value in testing_metrics.items():
        print(f"{metric}: {value}")

# Run the training and evaluation
train_and_evaluate()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 1844 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1567        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.006843879 |
|    clip_fraction        | 0.0286      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -4.07       |
|   

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.14e+03    |
|    ep_rew_mean          | 19.6        |
| time/                   |             |
|    fps                  | 1354        |
|    iterations           | 12          |
|    time_elapsed         | 18          |
|    total_timesteps      | 24576       |
| train/                  |             |
|    approx_kl            | 0.008893795 |
|    clip_fraction        | 0.0889      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.936      |
|    explained_variance   | 0.433       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.00932     |
|    n_updates            | 110         |
|    policy_gradient_loss | -0.00802    |
|    value_loss           | 0.0111      |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.14e+

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 7.14e+03     |
|    ep_rew_mean          | 37.4         |
| time/                   |              |
|    fps                  | 1341         |
|    iterations           | 22           |
|    time_elapsed         | 33           |
|    total_timesteps      | 45056        |
| train/                  |              |
|    approx_kl            | 0.0068851635 |
|    clip_fraction        | 0.0796       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.652       |
|    explained_variance   | 0.556        |
|    learning_rate        | 0.0003       |
|    loss                 | -0.012       |
|    n_updates            | 210          |
|    policy_gradient_loss | -0.00316     |
|    value_loss           | 0.00951      |
------------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean 

------------------------------------
| time/                 |          |
|    fps                | 959      |
|    iterations         | 1000     |
|    time_elapsed       | 5        |
|    total_timesteps    | 5000     |
| train/                |          |
|    entropy_loss       | -1.08    |
|    explained_variance | -24.3    |
|    learning_rate      | 0.0007   |
|    n_updates          | 999      |
|    policy_loss        | -0.0694  |
|    value_loss         | 0.00653  |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 960      |
|    iterations         | 1100     |
|    time_elapsed       | 5        |
|    total_timesteps    | 5500     |
| train/                |          |
|    entropy_loss       | -0.895   |
|    explained_variance | 0.0584   |
|    learning_rate      | 0.0007   |
|    n_updates          | 1099     |
|    policy_loss        | -0.043   |
|    value_loss         | 0.00353  |
-

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 7.14e+03  |
|    ep_rew_mean        | 4.2       |
| time/                 |           |
|    fps                | 958       |
|    iterations         | 2400      |
|    time_elapsed       | 12        |
|    total_timesteps    | 12000     |
| train/                |           |
|    entropy_loss       | -1.02     |
|    explained_variance | -1.54e+03 |
|    learning_rate      | 0.0007    |
|    n_updates          | 2399      |
|    policy_loss        | 0.0482    |
|    value_loss         | 0.00231   |
-------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.14e+03 |
|    ep_rew_mean        | 4.2      |
| time/                 |          |
|    fps                | 958      |
|    iterations         | 2500     |
|    time_elapsed       | 13       |
|    total_timesteps    | 12500    |
| train/             

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.14e+03 |
|    ep_rew_mean        | 2.4      |
| time/                 |          |
|    fps                | 960      |
|    iterations         | 3700     |
|    time_elapsed       | 19       |
|    total_timesteps    | 18500    |
| train/                |          |
|    entropy_loss       | -1.08    |
|    explained_variance | 0.118    |
|    learning_rate      | 0.0007   |
|    n_updates          | 3699     |
|    policy_loss        | -0.0375  |
|    value_loss         | 0.00197  |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.14e+03 |
|    ep_rew_mean        | 2.4      |
| time/                 |          |
|    fps                | 960      |
|    iterations         | 3800     |
|    time_elapsed       | 19       |
|    total_timesteps    | 19000    |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.14e+03 |
|    ep_rew_mean        | 3.32     |
| time/                 |          |
|    fps                | 961      |
|    iterations         | 5000     |
|    time_elapsed       | 26       |
|    total_timesteps    | 25000    |
| train/                |          |
|    entropy_loss       | -1.03    |
|    explained_variance | -1.84    |
|    learning_rate      | 0.0007   |
|    n_updates          | 4999     |
|    policy_loss        | -0.0579  |
|    value_loss         | 0.00406  |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.14e+03 |
|    ep_rew_mean        | 3.32     |
| time/                 |          |
|    fps                | 961      |
|    iterations         | 5100     |
|    time_elapsed       | 26       |
|    total_timesteps    | 25500    |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.14e+03 |
|    ep_rew_mean        | 3.52     |
| time/                 |          |
|    fps                | 962      |
|    iterations         | 6300     |
|    time_elapsed       | 32       |
|    total_timesteps    | 31500    |
| train/                |          |
|    entropy_loss       | -0.532   |
|    explained_variance | -3.91    |
|    learning_rate      | 0.0007   |
|    n_updates          | 6299     |
|    policy_loss        | 6.69e-05 |
|    value_loss         | 3.14e-05 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.14e+03 |
|    ep_rew_mean        | 3.52     |
| time/                 |          |
|    fps                | 962      |
|    iterations         | 6400     |
|    time_elapsed       | 33       |
|    total_timesteps    | 32000    |
| train/                |          |
|

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 7.14e+03  |
|    ep_rew_mean        | 3.37      |
| time/                 |           |
|    fps                | 964       |
|    iterations         | 7600      |
|    time_elapsed       | 39        |
|    total_timesteps    | 38000     |
| train/                |           |
|    entropy_loss       | -0.659    |
|    explained_variance | -1.81     |
|    learning_rate      | 0.0007    |
|    n_updates          | 7599      |
|    policy_loss        | -0.000235 |
|    value_loss         | 0.000118  |
-------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.14e+03 |
|    ep_rew_mean        | 3.37     |
| time/                 |          |
|    fps                | 964      |
|    iterations         | 7700     |
|    time_elapsed       | 39       |
|    total_timesteps    | 38500    |
| train/             

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.14e+03 |
|    ep_rew_mean        | 3.39     |
| time/                 |          |
|    fps                | 966      |
|    iterations         | 8900     |
|    time_elapsed       | 46       |
|    total_timesteps    | 44500    |
| train/                |          |
|    entropy_loss       | -0.00908 |
|    explained_variance | -5.96    |
|    learning_rate      | 0.0007   |
|    n_updates          | 8899     |
|    policy_loss        | 1.43e-07 |
|    value_loss         | 1.11e-07 |
------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 7.14e+03  |
|    ep_rew_mean        | 3.39      |
| time/                 |           |
|    fps                | 965       |
|    iterations         | 9000      |
|    time_elapsed       | 46        |
|    total_timesteps    | 45000     |
| train/                |    